<a href="https://colab.research.google.com/github/Jaizxzx/Neural-Network_and_Deep-Learning/blob/main/PyTorch/Multi_Class_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import torch
import torchvision
import torch.nn as nn
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
print(torch.__version__)

2.3.0+cu121


In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [45]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load full training set
full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Split training set into train and validation
train_size = int(0.8 * len(full_train_dataset))
train_dataset = random_split(full_train_dataset, [train_size, len(full_train_dataset) - train_size])[0]
# Load test set
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Function to separate features and labels
def separate_features_labels(dataset):
    dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    features, labels = next(iter(dataloader))
    return features, labels

# Separate features and labels for each set
train_features, train_labels = separate_features_labels(train_dataset)
test_features, test_labels = separate_features_labels(test_dataset)

# Print shapes to verify
print("Train features shape:", train_features.shape)
print("Train labels shape:", train_labels.shape)
print("Test features shape:", test_features.shape)
print("Test labels shape:", test_labels.shape)

Train features shape: torch.Size([48000, 1, 28, 28])
Train labels shape: torch.Size([48000])
Test features shape: torch.Size([10000, 1, 28, 28])
Test labels shape: torch.Size([10000])


###Model

In [46]:
class digit_model(nn.Module):
    def __init__(self):
        super(digit_model, self).__init__()
        self.l1 = nn.Linear(28*28, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.l2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.l3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.l4 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(x.shape[0], 28*28)
        x = self.relu(self.bn1(self.l1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.l2(x)))
        x = self.dropout(x)
        x = self.relu(self.bn3(self.l3(x)))
        x = self.dropout(x)
        x = self.l4(x)
        return x



In [47]:
model = digit_model().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

### Training Loop

In [48]:
def train_epoch(model, train_loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return total_loss / len(train_loader), 100. * correct / total


In [49]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = 100. * correct / total
    conf_matrix = confusion_matrix(all_labels, all_preds)
    precision, recall, _, _ = precision_recall_fscore_support(all_labels, all_preds, average=None)
    return accuracy, conf_matrix, precision, recall


In [50]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

best_val_acc = 0
patience = 5
epochs_no_improve = 0


In [51]:
for epoch in range(5):  # Increased max epochs
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, loss_fn)
    val_acc, _, _, _ = evaluate(model, val_loader)
    scheduler.step(train_loss)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print("Early stopping!")
            break

# Load best model and evaluate on test set
model.load_state_dict(torch.load('best_model.pth'))
test_acc, conf_matrix, precision, recall = evaluate(model, test_loader)
print(f"\nTest Accuracy: {test_acc:.2f}%")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nPrecision for each class:")
print(precision)
print("\nRecall for each class:")
print(recall)

Epoch 1: Train Loss: 0.6091, Train Acc: 85.32%, Val Acc: 94.89%
Epoch 2: Train Loss: 0.2565, Train Acc: 92.85%, Val Acc: 96.29%
Epoch 3: Train Loss: 0.2069, Train Acc: 94.11%, Val Acc: 96.60%
Epoch 4: Train Loss: 0.1747, Train Acc: 95.00%, Val Acc: 97.28%
Epoch 5: Train Loss: 0.1604, Train Acc: 95.42%, Val Acc: 97.27%

Test Accuracy: 97.23%

Confusion Matrix:
[[ 969    1    1    1    0    1    4    1    1    1]
 [   0 1124    4    0    1    1    1    1    3    0]
 [   3    1  998    4    3    0    1   13    9    0]
 [   0    0    4  984    0    2    0    9   10    1]
 [   1    0    4    0  953    0    5    3    1   15]
 [   4    0    0   13    1  849    6    2   12    5]
 [   4    3    1    0    4    5  935    0    6    0]
 [   1    5    8    0    0    0    0 1006    0    8]
 [   5    3    6    3    3    4    3    7  937    3]
 [   3    4    0    7    9    1    1    9    7  968]]

Precision for each class:
[0.97878788 0.98510079 0.97270955 0.97233202 0.97843943 0.98377752
 0.97803347 0